In [1]:
import random
import json
import pickle
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
lemmatizer = WordNetLemmatizer()

In [3]:
intents = json.loads(open("intents.json").read())

In [4]:
words = []
classes = []
documents = []
ignoreLetters = ["?", "!", ",", "."] 

In [5]:
for intent in intents['intents']:
  for pattern in intent['patterns']:
    wordList = nltk.word_tokenize(pattern)
    words.extend(wordList)
    documents.append((wordList, intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [6]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]

In [7]:
words = sorted(set(words))
classes = sorted(set(classes))

In [8]:
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

In [9]:
training = [] 

In [10]:
opempty = [0] * len(classes)

In [11]:
for documents in documents:
  bag = []
  wordPatterns = documents[0]
  wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
  for word in words: 
    bag.append(1) if word in wordPatterns else bag.append(0)

  oprow = list(opempty)
  oprow[classes.index(documents[1])] = 1
  training.append([bag, oprow])  

In [12]:
random.shuffle(training)

In [13]:
training = np.array(training)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [14]:
X_train = list(training[:, 0])
y_train = list(training[:, 1])

In [15]:
model = Sequential()
model.add(Dense(128, input_shape = (len(X_train[0]),), activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation = "softmax"))

In [16]:
sgd = SGD(learning_rate = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

In [17]:
ist = model.fit(np.array(X_train), np.array(y_train), epochs = 200, batch_size = 5, verbose = 1)

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/200
1/1 [==============================] - 0s 12ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/200
1/1 [==============================] - 0s 13ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/200
1/1 [==============================] - 0s 12ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/200
1/1 [==============================] - 0s 13ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/200
1/1 [==============================] - 0s 18ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/200
1/1 [==============================] - 0s 12ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/200
1/1 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/200
1/1 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/200
1/1 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - a

In [18]:
model.save("chatbot_model.h5", ist)
print("Done")

Done
